#### Imports

In [1]:
import Model_Blocks.Transformer as tfm
import Data_handler as dh
import Training as tr
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch
import os
from Utils import *
from text_gen import translate_sentence

#### Data management

In [2]:
df, vec_en, vec_fr, max_len_en, max_len_fr = dh.prepare_data()
padding_fr = vec_fr.vocabulary_['PPPading']
padding_en = vec_en.vocabulary_['PPPading']


len_fr_vocab = len(vec_fr.vocabulary_)
len_en_vocab = len(vec_en.vocabulary_)

train_dataset,test_dataset = train_test_split(df, test_size=0.16, random_state=23)

train_dataset = dh.VocabDataset(train_dataset, padding_fr, padding_en)
test_dataset = dh.VocabDataset(test_dataset, padding_fr, padding_en)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)





#### Model management

In [3]:
model = tfm.Transformer(len_en_vocab, len_fr_vocab)

In [4]:
#tr.train_model(model, train_loader, test_loader, optimizer=torch.optim.Adam(model.parameters()), criterion= torch.nn.CrossEntropyLoss(), padding_fr=padding_fr, padding_en=padding_en)
model.load_state_dict(torch.load('model_final.pt', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

<All keys matched successfully>

In [5]:
model.load_state_dict(torch.load('model_temp_training.pt', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

<All keys matched successfully>

In [6]:
#saving the model
#torch.save(model.state_dict(), 'model_final.pt')

In [7]:
print("Hello" in vec_en.vocabulary_)
print(vec_en.vocabulary_.get("Hello"))

True
1220


#### Testing the model

In [8]:
test_sentence = "I am eating a building with my mouth and I am eating a cat also I eat chinese food and people."
translation = translate_sentence(model, vec_en, vec_fr, test_sentence, max_len_fr, max_len_en, device=(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')))
print(f'Translation: {translation}')

RuntimeError: Expected all tensors to be on the same device, but got index is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA__index_select)